<a href="https://colab.research.google.com/github/S-anne-johnson/Mouse_census/blob/main/Census.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mouse Census

This is a Python script to automate the mouse census. After going to the mousehouse and recording cage information, the first thing you will need to do is make the spreadsheets that show the cages currently on the first and third floors. Our lab adds these as tabs on the same google spreadsheet [(titled Mouse House)](https://docs.google.com/spreadsheets/d/1du2879tqfgyZAD2Ryp1GM2I81_5zWhhDboRwqrCg86M/edit#gid=1009663345). When you make the sheets, a few things must be done for the code to run correctly.

1. The rack distributions for the 1st and 3rd floors, which are also tabs on the Mouse House google spreadsheet, need to be perfectly lined up with the sheets that show the current cages for those floors. This is because the cell locations on the rack distribution sheets are applied to the corresponding census sheets. You can copy and paste or duplicate the distribution sheets, and subsequently fill in the month's cages. Just make sure all the cells are in the exact same position.

2. Please do not change how the rack distribution sheets are formatted. For example, column B needs to have the cage numbers for the code to work.

3. The following are acceptable values to write on the sheets that show current cages:

        Name (B)

        Name (B)L

        Name (V)

        Name (B) NCC

        Name (B)L NCC

        Name (V) NCC

        Empty

        New Arrival

  Be sure not to change any spaces or characters on these values. The B(L) should have no space between the (B) and the L! Write all empty spaces as "Empty" if they count towards the room's unnocupied spots: **do not** write 'Empty' for any unused spots on the Yildrim rack, since this does not count towards Eroglu lab unnocupied space.

4. Please do not put extra text on the sheets that show current cages. If you would like to add details or notes that the above values do not include, do so after you have already run the code and generated the report.


The rack distributions for the 1st and 3rd floors and the completed spreadsheets that show the current cages on the 1st and 3rd floors need to be exported all as separate excel files ending in .xlsx . To download each, right click on the tab with its name and select Copy to > New spreadsheet. Open the spreadsheet and then go to File > Download > Microsoft Excel (.xlsx). Save them anywhere on your computer with the following names, using the current month and year:

    Rack Distribution Floor 1 Jan 2023.xlsx
    Jan 2023 Floor 1.xlsx
    Rack Distribution Floor 3 Jan 2023.xlsx
    Jan 2023 Floor 3.xlsx

Go onto mLIMS and below the 'Litters' tab, click 'Review Litters'. This lists all the active litters for the lab. Select 'Export' in the top bar and save the resulting spreadsheet:

    Litters Jan 2023.xlsx

***Go to the part of the code that says from openpyxl import load_workbook  and write in the names of these files in the corresponding places.***


Note of caution: Google Colab saves files that you import. You can view them by clicking on the folder icon on the left bar and going to content. If you need to re-run the code for any reason, you need to either use files with new names or delete the original files that you imported. If you use a file with the same name, without deleting the original file that was saved, the new file will have a (1), (2) etc. at the end because of the duplicate name. The code will then continue to use the old file instead of the new one because you haven't told it any different!


**Run the code by going to Runtime and clicking Run all.**

Once the report is generated, you can copy and paste it onto the [lab's census google document](https://docs.google.com/document/d/1pzdAonXRPd7Q8f65SXG2ukjZWeDXSeEafMngEfcrU8A/edit). After that, you're done!

In [ ]:
from google.colab import files  #click Choose Files and upload the first floor rack distribution spreadsheet
uploaded = files.upload()

Saving Rack Distribution Floor 1 Sep 2023.xlsx to Rack Distribution Floor 1 Sep 2023.xlsx


In [ ]:
from google.colab import files #upload the spreadsheet with the current cages for the first floor
uploaded = files.upload()

Saving Sep 2023 Floor 1.xlsx to Sep 2023 Floor 1.xlsx


In [ ]:
from google.colab import files #upload the third floor rack distribution spreadsheet
uploaded = files.upload()

Saving Rack Distribution Floor 3 Sep 2023.xlsx to Rack Distribution Floor 3 Sep 2023.xlsx


In [ ]:
from google.colab import files #upload the spreadsheet with the current cages for the third floor
uploaded = files.upload()

Saving Sep 2023 Floor 3.xlsx to Sep 2023 Floor 3.xlsx


In [ ]:
from google.colab import files #upload the spreadsheet with all the current active litters for the lab
uploaded = files.upload()

Saving Litters Sep 2023.xlsx to Litters Sep 2023.xlsx


We will use openpyxl, a library for reading and editing Excel files, to pull info from these sheets. For more information about openpyxl, see  [this helpful article](https://realpython.com/openpyxl-excel-spreadsheets-python/#getting-started-with-openpyxl).

In [ ]:
from openpyxl import load_workbook

#Write in the names for the four files you just uploaded
First_floor_rack_distribution = load_workbook(filename = "Rack Distribution Floor 1 Sep 2023.xlsx")
rack_distr_floor1 = First_floor_rack_distribution.active

Current_first_floor_cages = load_workbook(filename = "Sep 2023 Floor 1.xlsx")
current_cages_floor1 = Current_first_floor_cages.active

Third_floor_rack_distribution = load_workbook(filename = "Rack Distribution Floor 3 Sep 2023.xlsx")
rack_distr_floor3 = Third_floor_rack_distribution.active

Current_third_floor_cages = load_workbook(filename = "Sep 2023 Floor 3.xlsx")
current_cages_floor3 = Current_third_floor_cages.active

Litter_spreadsheet = load_workbook(filename = "Litters Sep 2023.xlsx")
litter_sheet = Litter_spreadsheet.active


import datetime #this will be used later to identify litters that were born recently

from datetime import date


Here we make a function that gives us a list of researchers we will need census data for. If a cell's value includes " (V)" or " (B)" and what's written before " (V)" or " (B)" is not already in the list, whatever is written previous to " (V)" or " (B)" will be added as a name. This will be done for both the rack distribution sheet (variable sheet1) and the current cage layout sheet (variable sheet2).

In [ ]:
def find_researcher(x,sheet1,sheet2):
    x = []
    for row in sheet1["A1:Z300"]:
        for cell in row:
            if " (V)" in str(cell.value) and str(cell.value[:str(cell.value).find(" (V)")]) not in x:
                x.append(cell.value[:str(cell.value).find(" (V)")])
            elif " (B)" in str(cell.value) and str(cell.value[:str(cell.value).find(" (B)")]) not in x:
                x.append(cell.value[:str(cell.value).find(" (B)")])
    for row in sheet2["A1:Z300"]:
        for cell in row:
            if " (V)" in str(cell.value) and str(cell.value[:str(cell.value).find(" (V)")]) not in x:
                x.append(cell.value[:str(cell.value).find(" (V)")])
            elif " (B)" in str(cell.value) and str(cell.value[:str(cell.value).find(" (B)")]) not in x:
                x.append(cell.value[:str(cell.value).find(" (B)")])
    return x

We define the census function which aquires census information and prints it out for a given researcher name.


In [ ]:
def census(name,sheet1,sheet2):

    virginspace = []
    breederspace = []

#Acquire a list of cell names style "A1","A2","A3" etc. for spots assigned to the researcher on the rack distribution
    def get_space(name, V_or_B, cell_list):
        for row in sheet1["A1:Z300"]:
            for cell in row:
                if cell.value != None and str(name) in str(cell.value) and str(V_or_B) in str(cell.value): #.value gives what is written inside a cell
                    spot = str(cell) #instead of giving the value, this will give a name of the cell which includes its position. For example, "<Cell 'Current Floor 3'.C10>"
                    startindex = spot.find(".")
                    endindex = spot.find(">")
                    spotlocation = spot[startindex+1:endindex] #For example,"C10"
                    cell_list.append(spotlocation)

    get_space(name, "(V)", virginspace)
    get_space(name, "(B)", breederspace)


    v = 0   #Get number of virgin cages
    for row in sheet2["A1:Z300"]:
        for cell in row:
            if str(name) in str(cell.value) and "(V)" in str(cell.value):
                v += 1

    b = 0   #Get number of breeder cages
    for row in sheet2["A1:Z300"]:
        for cell in row:
            if str(name) in str(cell.value) and "(B)" in str(cell.value):
                b += 1

    e = 0   #Get number of empty cages in virgin space
    for cell in virginspace:
        if "Empty" in str(sheet2[cell].value):
            e += 1

    E = 0   #Get number of empty cages in breeder space
    for cell in breederspace:
        if "Empty" in str(sheet2[cell].value):
            E += 1

    def get_cage_numbers(name,info, litter_or_NCC):
        cagelist = []
        for row in sheet2["A1:Z300"]:
            for cell in row:
                if str(name) in str(cell.value) and litter_or_NCC in str(cell.value):
                    spot = str(cell)
                    startindex = spot.find(".")
                    endindex = spot.find(">")
                    spotlocation = spot[startindex+1:endindex]
                    rownumber = spot[startindex+2:endindex]
                    #Column B has cage numbers(ex. O101) for the first cage of every row
                    if str(sheet2["B"+rownumber].value)[0] == "O":
                        n = int(sheet2["B"+rownumber].value[1:])-1
                    else:
                        n = int(sheet2["B"+rownumber].value) - 1
                    for item in sheet2["C"+rownumber:spotlocation]:
                        for itemm in item:
                            n +=1
                        if len(str(n))==3:
                            info = info + " 0" + str(n) + ","
                            if "litters" in info:
                                cagelist.append("0" + str(n))
                        else:
                            info = info + " " + str(n) + ","
                            if "litters" in info:
                                cagelist.append(str(n))
        if info[-1] != ",": #Don't print anything if there were no cages
            print("")
        else:
            print(info[:-1]+".")





        breeding_cages = []

        for cell in litter_sheet["C"]:
          if cell.value != None:
            cell_ = str(cell.value)
            spot_ = str(cell)
            startindex_ = spot_.find(".") + 2
            endindex_ = spot_.find(">")
            birthdate =  litter_sheet["D"+spot_[startindex_:endindex_]]
            if type(birthdate.value) == datetime.datetime:
                birth = birthdate.value
                time_passed = date.today() - birth.date()
                if time_passed.days < 35:

                    if cell.value != None:
                        if "O" in cell_:
                            cell_ = cell_.replace("O","0")
                        if "_" in cell_:
                            cell_ = cell_[:cell_.find("_")]
                        if "-" in cell_:
                            cell_ = cell_[:cell_.find("-")]
                        breeding_cages.append(cell_)


        pups_cages = []

        for cell in litter_sheet["E"]:
            cell_ = str(cell.value)
            spot_ = str(cell)
            startindex_ = spot_.find(".") + 2
            endindex_ = spot_.find(">")
            birthdate =  litter_sheet["D"+spot_[startindex_:endindex_]]
            if type(birthdate.value) == datetime.datetime:
                birth = birthdate.value
                time_passed = date.today() - birth.date()
                if time_passed.days < 35:


                    if cell.value != None:
                        if "," in cell_:
                            for cagename in cell_.split(", "):
                                if "O" in cagename:
                                    cagename = cagename.replace("O","0")
                                if "_" in cagename:
                                    cagename = cagename[:cagename.find("_")]
                                if "-" in cagename:
                                    cagename = cagename[:cagename.find("-")]
                                pups_cages.append(cagename)
                        else:
                            if "O" in cell_:
                                cell_ = cell_.replace("O","0")
                            if "_" in cell_:
                                cell_ = cell_[:cell_.find("_")]
                            if "-" in cell_:
                                cell_ = cell_[:cell_.find("-")]
                            pups_cages.append(cell_)

        not_on_mlims = ""
        if cagelist != []:
            for cage in cagelist:
                    if cage not in breeding_cages and cage not in pups_cages:
                        not_on_mlims = not_on_mlims +" " +cage + ","
            if not_on_mlims.count(",") == 1:
                print("Litter in cage" +not_on_mlims[:-1] +" NOT on mlims")
            if not_on_mlims.count(",") > 1:
                print("Litters in cages" +not_on_mlims[:-1]+" NOT on mlims")



    print(str(v) +" virgin cages out of " +str(len(virginspace)) +" total virgin spots")
    print(str(e) +" empty spots in virgin space")
    print(str(b) +" breeder cages out of " +str(len(breederspace)) +" total breeder spots")
    print(str(E) +" empty spots in breeder space")
    get_cage_numbers(name,"Cages with litters:","(B)L")
    get_cage_numbers(name,"Cages with no mlims cage card:","NCC")



We obtain the total number of occupied cages by summing up the number of cages of each researcher in the researcher list and adding the number of New Arrivals. We also get the number of empty cages. Finally, we iterate over the researcher list to print out census info for each researcher.

In [ ]:

#Get the total numbers of occupied and empty cages on the first floor

t=0
for person in find_researcher('x', rack_distr_floor1, current_cages_floor1):
    for row in current_cages_floor1["A1:Z300"]:
        for cell in row:
            if str(person) in str(cell.value):
                t += 1

n=0
for row in current_cages_floor1["A1:Z300"]:
    for cell in row:
        if "New Arrival" in str(cell.value):
            n += 1

t = t + n

empty=0
for row in current_cages_floor1["A1:Z300"]:
    for cell in row:
        if "Empty" in str(cell.value):
            empty += 1

print("There are " +str(t) +" total occupied spots on the first floor and " +str(empty) +" empty. There are 552 total spots on the Eroglu racks. " +str(n) +" of the cages are New Arrivals.\n")


#Get total numbers of occupied and empty cages on third floor
T=0
for person in find_researcher('x', rack_distr_floor3, current_cages_floor3):
    for row in current_cages_floor3["A1:Z300"]:
        for cell in row:
            if str(person) in str(cell.value):
                T += 1

N=0
for row in current_cages_floor3["A1:Z300"]:
    for cell in row:
        if "New Arrival" in str(cell.value):
            N += 1

T = T + N

Empty=0
for row in current_cages_floor3["A1:Z300"]:
    for cell in row:
        if "Empty" in str(cell.value):
            Empty += 1

print("There are " +str(T) +" total occupied spots on the third floor and " +str(Empty) +" empty out of 138 total spots. " +str(N) +" of the cages are New Arrivals.\n")



print("\nFirst Floor \n")  #print census information for all the first floor by iterating over the first floor researcher list

for person in find_researcher('x', rack_distr_floor1, current_cages_floor1):
     print(person +":")
     census(person, rack_distr_floor1, current_cages_floor1)
     print("\n")



print("\nThird Floor\n") #print census information for all the third floor by iterating over the third floor researcher list

for person in find_researcher('x', rack_distr_floor3, current_cages_floor3):
     print(person +":")
     census(person, rack_distr_floor3, current_cages_floor3)
     print("\n")






There are 370 total occupied spots on the first floor and 204 empty. There are 552 total spots on the Eroglu racks. 2 of the cages are New Arrivals.

There are 73 total occupied spots on the third floor and 65 empty out of 138 total spots. 0 of the cages are New Arrivals.


First Floor 

Shiyi:
16 virgin cages out of 35 total virgin spots
3 empty spots in virgin space
26 breeder cages out of 17 total breeder spots
0 empty spots in breeder space
Cages with litters: 0121, 0124, 2227, 2233, 2234, 2235, 2238.
Litters in cages 0121, 2227, 2233, 2235 NOT on mlims



Fra:
7 virgin cages out of 24 total virgin spots
13 empty spots in virgin space
7 breeder cages out of 6 total breeder spots
0 empty spots in breeder space
Cages with litters: 2264, 2269.



Juan:
15 virgin cages out of 21 total virgin spots
4 empty spots in virgin space
22 breeder cages out of 21 total breeder spots
3 empty spots in breeder space
Cages with litters: 3132, 3134, 3136, 3137, 3138, 3141, 3142, 3144, 3147, 3149.
Cag